In [2]:
spark.sql("""
    SELECT count(tmp.id) usercount,tmp.create_time
    FROM 
        (SELECT date_format(create_time,'yyyy-MM-dd')as create_time, id
        FROM `banda-etl-s3`.t_customer ) tmp   
    GROUP BY  tmp.create_time
    having tmp.create_time>  '2021-01-01'
    order by tmp.create_time desc
""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-----------+
|usercount|create_time|
+---------+-----------+
|     3214| 2021-04-23|
|     7774| 2021-04-22|
|     7230| 2021-04-21|
|     7570| 2021-04-20|
|     7403| 2021-04-19|
|     7055| 2021-04-18|
|     7840| 2021-04-17|
|     7464| 2021-04-16|
|     8031| 2021-04-15|
|     8826| 2021-04-14|
|     7157| 2021-04-13|
|     7301| 2021-04-12|
|     6491| 2021-04-11|
|     7075| 2021-04-10|
|     8621| 2021-04-09|
|     8455| 2021-04-08|
|     7738| 2021-04-07|
|     6964| 2021-04-06|
|     7649| 2021-04-05|
|     5815| 2021-04-04|
+---------+-----------+
only showing top 20 rows

In [ ]:
spark.sql("""select  * from `banda-etl-s3`.t_customer limit 100""").show()

In [1]:
sc.install_pypi_package("boto3")
sc.install_pypi_package("pandas")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1619491650839_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/15/f7/a80db6492ca4c9471c474f88853f2864766b136ff30240a61efcea965c14/boto3-1.17.58-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/63/d0/693477c688348654ddc21dcdce0817653a294aa43f41771084c25e7ff9c7/s3transfer-0.4.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4f/c0/f42b171b59f5001aa3754bcebe379fb14065b057645ab89a029d831a48b1/botocore-1.20.58-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/09/c6/d3e3abe5b4f4f16cf0dfc9240ab7ce10c2baa0e268989a4e3ec19e90c84e/urllib3-1.26.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl


In [41]:
import boto3
jobs={}
# 先获取文件路径，拿文件内容
from botocore.client import Config
s3 = boto3.resource('s3', config=Config(signature_version='s3v4'))
def getsqlPathList():
    bucket='rupiahplus-configs'
    sqlpath='etl/sparksql/daily/'
    s3bucket = s3.Bucket(bucket)
    sql_list = []
    for obj in s3bucket.objects.filter(Prefix=sqlpath):
        file=obj.key.split("/")[-1]
        filemain= file.split(".")
        if len(filemain)!=2:
            continue
        print(filemain)
        fileNm=filemain[0]
        fileType=filemain[1]
        if fileType is None:
            pass
        elif fileType=='txt' or fileType=='sql':
            body=obj.get()['Body'].read().decode('utf-8')
            jobs[fileNm]=body
        elif fileType=='py':
#             python文件就下载下来
            print(sqlpath+file)
            print('day/'+file)
            s3.Bucket(bucket).download_file(sqlpath+file, '/hello.txt')
#             with open('/day/'+file, 'wb') as data:
#                 s3.download_fileobj(bucket, sqlpath+file, data)
#         s3://rupiahplus-configs/etl/sparksql/daily/
#             s3.meta.client.download_file(bucket, sqlpath+file, '/tmp1/hello.txt')
#             body=obj.get()['Body'].read().decode('utf-8')
#             jobs[fileNm]=body
    print(jobs)
    return jobs
# class Job:
#     def run(name,sql):
#         jobs[name]=spark.sql(sql)
# job = new Job()

# job.run()
jobs=getsqlPathList()
# print(jobs)
temp=None
for i in jobs:
    print(jobs[i])
    if temp is None:
        temp=spark.sql(jobs[i])
#     else:
        temp=temp.join(spark.sql(jobs[i]),how='full',on='create_time')
# temp.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Errno 13] Permission denied: '/hello.txt.C46D1B04'
Traceback (most recent call last):
  File "<stdin>", line 28, in getsqlPathList
  File "/tmp/1619404925736-0/lib/python3.7/site-packages/boto3/s3/inject.py", line 246, in bucket_download_file
    ExtraArgs=ExtraArgs, Callback=Callback, Config=Config)
  File "/tmp/1619404925736-0/lib/python3.7/site-packages/boto3/s3/inject.py", line 172, in download_file
    extra_args=ExtraArgs, callback=Callback)
  File "/tmp/1619404925736-0/lib/python3.7/site-packages/boto3/s3/transfer.py", line 307, in download_file
    future.result()
  File "/tmp/1619404925736-0/lib/python3.7/site-packages/s3transfer/futures.py", line 106, in result
    return self._coordinator.result()
  File "/tmp/1619404925736-0/lib/python3.7/site-packages/s3transfer/futures.py", line 265, in result
    raise self._exception
  File "/tmp/1619404925736-0/lib/python3.7/site-packages/s3transfer/tasks.py", line 126, in __call__
    return self._execute_main(kwargs)
  File "/tmp/16

In [6]:

import pyspark
import sys
import os
import argparse
import importlib
from pyspark.sql import SparkSession
spark = SparkSession\
        .builder\
        .appName("Python Demo")\
        .config("hive.metastore.client.factory.class", "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory") \
        .enableHiveSupport()\
        .getOrCreate()
if os.path.exists('python1.zip'):
    sys.path.insert(0, 'python1.zip')
else:
    sys.path.insert(0, './python1')

parser = argparse.ArgumentParser()
# parser.add_argument('--job', type=str, required=True)
# parser.add_argument('--job-args', nargs='*')
args = parser.parse_args()

# sc = pyspark.SparkContext(appName=args.job_name)
job_module = importlib.import_module('python2')
print(job_module.aaa())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

No module named 'python2'
Traceback (most recent call last):
  File "/tmp/1619509231910-0/lib64/python3.7/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1006, in _gcd_import
  File "<frozen importlib._bootstrap>", line 983, in _find_and_load
  File "<frozen importlib._bootstrap>", line 965, in _find_and_load_unlocked
ModuleNotFoundError: No module named 'python2'



In [1]:
df1=spark.sql("""
SELECT  cust.id AS id
FROM `banda-etl-s3`.t_customer cust
LEFT JOIN `banda-etl-s3`.t_test_customer test_cust
ON cust.id = test_cust.customer_id
WHERE test_cust.customer_id is null   

""")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1620282896169_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df2=spark.sql("""
     WITH loan_situation AS (
SELECT  loan_detail.*
       ,app.status                                                                                                 AS last_loan_status
       ,CASE WHEN app.status IN ( 'CURRENT','OVERDUE','PAID_OFF','GRACE_PERIOD' ) THEN pay.due_date  ELSE NULL END AS last_loan_due_date
       ,CASE WHEN app.status IN ( 'CURRENT','OVERDUE','GRACE_PERIOD' ) THEN COALESCE (pay.principal_accr,0)+ COALESCE (pay.interest_accr,0) + COALESCE (pay.default_accr,0) + COALESCE (pay.service_fee_accr,0)+ COALESCE (pay.default_fee_accr,0)-( COALESCE (pay.principal_paid,0)+ COALESCE (pay.interest_paid,0) + COALESCE (pay.default_paid,0) + COALESCE (pay.service_fee_paid,0)+ COALESCE (pay.default_fee_paid,0) )  ELSE NULL END AS current_loan_out
       ,CASE WHEN app.status IN ( 'CURRENT','OVERDUE','GRACE_PERIOD' ) THEN COALESCE(pay.principal_accr,0)- COALESCE(pay.principal_paid,0)  ELSE NULL END AS current_loan_principal
FROM 
(
	SELECT  app.customer_id
	       ,MIN(app.create_time)                                                        AS first_apply_time
	       ,MIN(cu_status.create_time)                                                  AS first_issue_time
	       ,MIN( If( app.loan_type = 'RE_PAYDAY',app.create_time,NULL ) )               AS first_repayday_create_time
	       ,COUNT( If( cu_status.id > 0 AND app.loan_type != 'ROLLOVER',app.id,NULL ) ) AS total_issue_time
	       ,SUM( If( cu_status.id > 0 AND app.loan_type != 'ROLLOVER',app.amount,0 ) )  AS total_issue_amount
	       ,SUM( If( cu_status.id > 0,COALESCE (pay.principal_paid,0)+ COALESCE (pay.interest_paid,0) + COALESCE (pay.default_paid,0) + COALESCE (pay.rollover_fee_paid,0)+ COALESCE (pay.service_fee_paid,0)- COALESCE (pay.erase_amount,0)- COALESCE (pay.coupon_amount,0),0 ) ) AS total_paid_amount
	       ,MAX(app.id)                                                                 AS last_loan_id
	FROM `banda-etl-s3`.t_loan_app app
	LEFT JOIN `banda-etl-s3`.t_loan_app_status_log cu_status
	ON app.id = cu_status.loan_app_id AND cu_status.new_status = 'CURRENT'
	LEFT JOIN `banda-etl-s3`.t_lpay pay
	ON app.id = pay.loan_app_id
	WHERE app.create_time + interval '7' hour > date '2019-08-01' 
	GROUP BY  app.customer_id 
) loan_detail
LEFT JOIN `banda-etl-s3`.t_loan_app app
ON loan_detail.last_loan_id = app.id
LEFT JOIN `banda-etl-s3`.t_lpay pay
ON loan_detail.last_loan_id = pay.loan_app_id ), record_detail AS (
SELECT  customer.id
       ,MIN(pi.create_time)                                                                             AS verified_time
       ,MIN( IF( pi.marital_status IS NOT NULL,pi.create_time,NULL ) )                                  AS pi_time
       ,MIN(emp.create_time)                                                                            AS emp_time
       ,MIN(contact.create_time)                                                                        AS contact_time
       ,MIN( IF( FILE.file_type = 'KTP_PHOTO',FILE.create_time,NULL ) )                                 AS ktp_time
       ,MIN( IF( FILE.file_type = 'KTP_HAND_PHOTO',FILE.create_time,NULL ) )                            AS ktp_hand_time
       ,MIN(tongdun.create_time)                                                                        AS increase_time
       ,MIN( IF( FILE.file_type IN( 'YITU_BIO_PACKAGE','FACEID_BIO_PACKAGE' ),FILE.create_time,NULL ) ) AS bio_time
FROM `banda-etl-s3`.t_customer customer
LEFT JOIN `banda-etl-s3`.t_record_personal_info pi
ON pi.customer_id = customer.id
LEFT JOIN `banda-etl-s3`.t_record_employment emp
ON emp.customer_id = customer.id
LEFT JOIN `banda-etl-s3`.t_record_contact contact
ON contact.customer_id = customer.id
LEFT JOIN `banda-etl-s3`.t_record_file FILE
ON FILE.customer_id = customer.id
LEFT JOIN `banda-etl-s3`.t_tongdun_data tongdun
ON tongdun.customer_id = customer.id
GROUP BY  customer.id )
         ,record AS (
SELECT  id
       ,CASE WHEN pi_time > emp_time AND pi_time > contact_time AND pi_time > file_time AND pi_time > file_h_time AND pi_time > bio_time THEN pi_time 
             WHEN emp_time > pi_time AND emp_time > contact_time AND emp_time > file_time AND emp_time > file_h_time AND emp_time > bio_time THEN emp_time 
             WHEN contact_time > pi_time AND contact_time > emp_time AND contact_time > file_time AND contact_time > file_h_time AND contact_time > bio_time THEN contact_time 
             WHEN file_time > pi_time AND file_time > emp_time AND file_time > contact_time AND file_time > file_h_time AND file_time > bio_time THEN file_time 
             WHEN bio_time > pi_time AND bio_time > emp_time AND bio_time > contact_time AND bio_time > file_time AND bio_time > file_h_time THEN bio_time  ELSE file_h_time END AS record_time
FROM 
(
	SELECT  customer.id
	       ,MIN(pi.create_time)                                                                             AS pi_time
	       ,MIN(emp.create_time)                                                                            AS emp_time
	       ,MIN(contact.create_time)                                                                        AS contact_time
	       ,MIN( IF( FILE.file_type = 'KTP_PHOTO',FILE.create_time,NULL ) )                                 AS file_time
	       ,MIN( IF( FILE.file_type = 'KTP_HAND_PHOTO',FILE.create_time,NULL ) )                            AS file_h_time
	       ,MIN( IF( FILE.file_type IN( 'YITU_BIO_PACKAGE','FACEID_BIO_PACKAGE' ),FILE.create_time,NULL ) ) AS bio_time
	FROM `banda-etl-s3`.t_customer customer
	LEFT JOIN `banda-etl-s3`.t_record_personal_info pi
	ON pi.customer_id = customer.id AND pi.marital_status != ''
	LEFT JOIN `banda-etl-s3`.t_record_employment emp
	ON emp.customer_id = customer.id
	LEFT JOIN `banda-etl-s3`.t_record_contact contact
	ON contact.customer_id = customer.id
	LEFT JOIN `banda-etl-s3`.t_record_file FILE
	ON FILE.customer_id = customer.id
	WHERE date( customer.create_time + interval '7' hour ) > date '2019-08-01' 
	AND pi.customer_id > 0 
	AND emp.customer_id > 0 
	AND contact.customer_id > 0 
	AND FILE.customer_id > 0 
	GROUP BY  customer.id 
) detail
WHERE file_time IS NOT NULL 
AND file_h_time IS NOT NULL 
AND bio_time IS NOT NULL ), first_loan AS ( 
SELECT  customer.id
       ,MIN(auto_loan.create_time) AS authorized_time
       ,MIN(app.create_time)       AS apply_time
       ,MIN(lg.create_time)        AS issue_time
       ,MIN(lgt.create_time)       AS withdraw_time
FROM `banda-etl-s3`.t_customer customer
LEFT JOIN `banda-etl-s3`.t_auto_review_loan auto_loan
ON customer.id = auto_loan.customer_id AND auto_loan.real_policy = 'payday_amount_scorecard' AND auto_loan.result = 'PASS'
LEFT JOIN `banda-etl-s3`.t_loan_app app
ON app.customer_id = customer.id AND app.loan_type = 'PAYDAY'
LEFT JOIN `banda-etl-s3`.t_loan_app_status_log lg
ON lg.loan_app_id = app.id AND lg.new_status = 'CURRENT'
LEFT JOIN `banda-etl-s3`.t_loan_app app1
ON app1.id = lg.loan_app_id AND app1.loan_type != 'ROLLOVER'
LEFT JOIN `banda-etl-s3`.t_loan_app_status_log lgt
ON lgt.loan_app_id = app.id AND lgt.old_status = 'AUTHORIZATION' AND lgt.new_status = 'FINAL_REVIEW'
GROUP BY  customer.id )
         ,customer_device AS (
SELECT  customer_id
       ,value
       ,create_time
FROM `banda-etl-s3`.`t_customer_device_info`
WHERE id IN ( SELECT id FROM  ( SELECT customer_id, MIN (id) AS id FROM `banda-etl-s3`.`t_customer_device_info` WHERE type = 'ANDROID_ID' GROUP BY customer_id  ) a ) ), version AS ( 
SELECT  android_id
       ,app_version
       ,MIN( cast( concat( partition_0,'-',partition_1,'-',partition_2 ) AS date ) ) AS version_time
FROM `ddb_event_track_s3`.`id_adapundi_event_track`
WHERE partition_0 >= '2021' 
AND app_version IS NOT NULL 
GROUP BY  android_id
         ,app_version )
         ,register_version AS (
SELECT  rk.*
       ,cu.create_time
FROM 
(
	SELECT  de.customer_id
	       ,de.value
	       ,de.create_time AS device_time
	       ,ve.version_time
	       ,ve.app_version
	       ,row_number() OVER ( partition BY de.customer_id ORDER BY ve.version_time DESC ) rnk
	FROM customer_device de
	LEFT JOIN version ve
	ON de.value = ve.android_id AND ve.version_time < de.create_time
	WHERE ve.android_id IS NOT NULL  
) rk
LEFT JOIN `banda-etl-s3`.`t_customer` cu
ON rk.customer_id = cu.id
WHERE cu.create_time > rk.version_time 
AND rk.rnk = 1
ORDER BY rk.customer_id ), change_mobile AS ( 
SELECT  customer_id
       ,first_cn_time
       ,CASE WHEN first_cn_time < first_cn_success_time THEN first_cn_success_time END AS first_cn_success_time
       ,CASE WHEN first_cn_time < first_cn_success_time THEN 1 
             WHEN first_cn_time IS NOT NULL THEN 0 END                                 AS ch_end
FROM 
(
	SELECT  customer_id
	       ,MIN( IF(tag_id = 10001,create_time,NULL) ) AS first_cn_time
	       ,MIN( IF(tag_id = 10002,create_time,NULL) ) AS first_cn_success_time
	FROM `banda-etl-s3`.`t_customer_tag_rel`
	WHERE tag_id IN (10001, 10002) 
	GROUP BY  customer_id 
) de )
SELECT  cus.id                                                                                                                                      AS id
       ,cus.create_time                                                                                                                             AS register_time
       ,cus.last_login_time
       ,cus.inviter_id
       ,CASE WHEN whitelist.customer_id > 0 THEN 'Y'  ELSE 'N' END                                                                                  AS is_whitelist
       ,install.af_channel
       ,install.media_source
       ,install.campaign
       ,install.agency
       ,install.fb_campaign_id
       ,install.channel_details_id
       ,install.install_time
       ,CASE WHEN record.id > 0 THEN 'Y'  ELSE 'N' END                                                                                              AS is_record
       ,record.record_time
       ,loan.first_apply_time
       ,loan.first_issue_time
       ,loan.first_repayday_create_time
       ,loan.total_issue_time
       ,loan.total_issue_amount
       ,loan.total_paid_amount
       ,loan.last_loan_status
       ,loan.last_loan_due_date
       ,loan.current_loan_out
       ,loan.current_loan_principal
       ,rd.verified_time
       ,rd.pi_time
       ,rd.emp_time
       ,rd.contact_time
       ,rd.bio_time
       ,CASE WHEN rd.ktp_time > cus.create_time AND rd.ktp_hand_time > cus.create_time AND rd.ktp_time > rd.ktp_hand_time THEN rd.ktp_time 
             WHEN rd.ktp_time > cus.create_time AND rd.ktp_hand_time > cus.create_time AND rd.ktp_time < rd.ktp_hand_time THEN rd.ktp_hand_time END AS ktp_time
       ,rd.increase_time
       ,record.record_time                                                                                                                          AS required_time
       ,fl.authorized_time
       ,fl.withdraw_time
       ,ve.app_version                                                                                                                              AS register_version
       ,ve.version_time                                                                                                                             AS register_version_time
       ,cus.main_account
       ,CASE WHEN cus.main_account IS NOT NULL THEN 'N'  ELSE 'Y' END                                                                               AS is_main_account
       ,cmo.ch_end
       ,cus.status
       ,cmo.first_cn_time
       ,cmo.first_cn_success_time
FROM `banda-etl-s3`.t_customer cus
LEFT JOIN `banda-etl-s3`.t_customer_install_info install
ON cus.id = install.customer_id
LEFT JOIN change_mobile cmo
ON cus.id = cmo.customer_id
LEFT JOIN `banda-etl-s3`.t_whitelist_old_customer whitelist
ON cus.id = whitelist.customer_id
LEFT JOIN loan_situation loan
ON cus.id = loan.customer_id
LEFT JOIN record
ON cus.id = record.id
LEFT JOIN record_detail rd
ON cus.id = rd.id
LEFT JOIN first_loan fl
ON cus.id = fl.id
LEFT JOIN register_version ve
ON cus.id = ve.customer_id
""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
cols = [df1[c] for c in df1.columns]
print(cols)
for df in [df2]:
    df1 = df1.join(df, df1.id == df.id, 'left')
    cols += [df[c] for c in df.columns if c != 'id']
# print(cols)
# df1.show()

# df1.select(cols).printSchema()



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Column<b'id'>]
+------+------+--------------------+--------------------+----------+------------+-----------+------------+--------------------+----------+--------------+------------------+--------------------+---------+-----------+--------------------+----------------+--------------------------+----------------+------------------+-----------------+----------------+------------------+----------------+----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------+-------------+-------------+---------------+-------------+----------------+---------------------+------------+---------------+------+-------------+-------------+---------------------+
|    id|    id|       register_time|     last_login_time|inviter_id|is_whitelist| af_channel|media_source|            campaign|    agency|fb_campaign_id|channel_details_id|        install_time|is_record|record_time|    first_apply_time|first_issue_time|first_repayday_create_t

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|      date|
+----------+
|2019-08-01|
|2019-08-02|
|2019-08-03|
|2019-08-04|
|2019-08-05|
|2019-08-06|
|2019-08-07|
|2019-08-08|
|2019-08-09|
|2019-08-10|
|2019-08-11|
|2019-08-12|
|2019-08-13|
|2019-08-14|
|2019-08-15|
|2019-08-16|
|2019-08-17|
|2019-08-18|
|2019-08-19|
|2019-08-20|
+----------+
only showing top 20 rows